In [1]:
import pandas as pd
from sklearn. feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

Q1. Downloading the data


We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

16
17
18
19


In [2]:
df_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_jan.shape[1]

19

Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59
42.59
52.59
62.59

In [3]:
# Duration is dropoff time - pickup time
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]

In [4]:
df_jan["duration"] = df_jan["duration"].dt.total_seconds() / 60
df_jan["duration"]

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [5]:
df_jan["duration"].std()

42.59435124195458

Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%
92%
95%
98%

In [6]:
length_before_dropping = df_jan["duration"].shape[0]
print('Length of total records:', length_before_dropping)


length_after_dropping_outliers = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)].shape[0]
df_jan = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)]
print('Length of total records after dropping outliers:', length_after_dropping_outliers)


print("fraction of the records left after you dropped the outliers:",  ((length_after_dropping_outliers)*100/length_before_dropping))

Length of total records: 3066766
Length of total records after dropping outliers: 3009173
fraction of the records left after you dropped the outliers: 98.1220282212598


Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

2
155
345
515
715

In [7]:
categorical = ["PULocationID", "DOLocationID"]
numerical = ["trip_distance"]

In [8]:
df_jan[categorical] = df_jan[categorical].astype(str)
train_dicts = df_jan[categorical].to_dict(orient = 'records')
# train_dicts

In [9]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [10]:
y_train = df_jan["duration"].values

Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

from sklearn.metrics import root_mean_squared_error
rmse = root_mean_squared_error(y_train, y_pred)
rmse

7.649261927686161

Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81

In [12]:
df_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')
df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
df_feb["duration"] = df_feb["duration"].dt.total_seconds() / 60
df_feb = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60)]

In [13]:
df_feb[categorical] = df_feb[categorical].astype(str)
val_dicts = df_feb[categorical].to_dict(orient = 'records')

X_val = dv.transform(val_dicts)
y_val = df_feb["duration"].values


y_pred = lr.predict(X_val)

rmse = root_mean_squared_error(y_val, y_pred)
rmse

7.811817957524739